<h1 align="center">     Simple Text Generator based on comments from Gossip Portal  </h1> 

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import numpy as np
from collections import Counter
from tensorflow.data import Dataset 
from tensorflow import expand_dims, squeeze
from tensorflow.random import categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GRU
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow import TensorShape

#Web Scraping


**It takes at least 1M characters to train a model based on RNN ~ 5000 comments** 

In [ ]:
page_url = 'https://www.pudelek.pl/bal-tvn-2022-czy-to-xena-wojownicza-ksiezniczka-nie-to-tylko-malgorzata-rozenek-w-zlotej-zbroi-zdjecia-6813461672434400a/'
page = requests.get(page_url)

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

In [ ]:
clas = "sc-1mskw74-0 sc-7eqdwf-0 sc-q1w81m-0 cyNurK"
comm = soup.find_all("div", {"class": clas})
print(comm[4].get_text())
print(len(comm))

Bardziej przypomina kostium syreny...
19


**A HTML file does not contain all comments, padding is inteactive and does not chcenge the URL adress, so there is a need to use API through GraphQL.**

In [ ]:
query = """query comments(
    $productId: String!
      $materialId: String!
        $parentId: String
          $offset: Int = 0
            $first: Int = 10
              $sort: CommentsSortType = LATEST

                  ) {
                      comments: comments(
                            productId: $productId
                                contentId: $materialId
                                    parentId: $parentId
                                        offset: $offset
                                            first: $first
                                                sort: $sort
                                                  ) {
                                                        ...commentEdges
                                                            pageInfo {
                                                                    hasNextPage
                                                                          hasPreviousPage
                                                                                endCursor
                                                                                      totalCount
                                                                                              }
                                                                                                 }

                                                                                                      }
                                                                                                                                  
                                                                                                   fragment commentEdges on CommentsConnection {
                                                                                                         edges {
                                                                                                             node {
                                                                                                                      text
                                                                                                                          nick
                                                                                                                               id
                                                                                                                                   replies
                                                                                                                                       }
                                                                                                                                          }
                                                                                                                                             }"""
#query return max 100 comments
main_comm = []
for off in range(0, 2500, 100):
  data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": "6813461672434400", "offset": off, "first": 100, "sort": "LATEST"}}
  response = requests.post('https://data-api.wp.pl/graphql', json=data)
  main_comm.append(response.json())
#json.dumps(response.json(), indent=2, ensure_ascii=False)
#print(str(main_comm[1])[-500:])

In [ ]:
#Dict to str
comments=''
for mc in main_comm:
    for i in range(0, len(mc['data']['comments']['edges'])):
      a = mc['data']['comments']['edges'][i]['node']

      text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
      nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

      comments = comments + text + nick

In [ ]:
#print(comments[:500])

In [ ]:
#number of main comments
comments.count('nick')

1642

In [ ]:
len(comments)

186625

**Getting replies to main comments**

In [ ]:
#getting id when replies > 0
parentId = [] 
for mc in main_comm:
  for i in range(0, len(mc['data']['comments']['edges'])):
    if mc['data']['comments']['edges'][i]['node']['replies']>0:
      parentId.append(mc['data']['comments']['edges'][i]['node']['id'])

In [ ]:
flag = 0
for id in parentId:
  data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": "6813461672434400", "parentId": id, "offset": 0, "first": 100, "sort": "LATEST"}}
  response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
  replies = response_rep.json()

  for i in range(0, len(replies['data']['comments']['edges'])):
    a = replies['data']['comments']['edges'][i]['node']

    text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
    nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

    comments = comments + text + nick

    if a['replies']>100:
      flag += 1

In [ ]:
#another round, if any main comment had more than 100 replies
if flag > 0:
  for id in parentId:
    data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": "6813461672434400", "parentId": id, "offset": 100, "first": 100, "sort": "LATEST"}}
    response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
    replies = response_rep.json()

    for i in range(0, len(replies['data']['comments']['edges'])):
      a = replies['data']['comments']['edges'][i]['node']

      text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
      nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

      comments = comments + text + nick

In [ ]:
len(comments)

255364

In [ ]:
print(comments[-500:])

g przydałby się pani Gosi....'
'nick': 'Bbhjj'

'text': 'tyle łapek w górę , tylko za same emotikonki , to chyba kupione!!!'
'nick': 'jak nic!😂😁'

'text': 'Bez tej zbroi byłoby znośnie..ale ona chyba lubi wyglądać przaśnie'
'nick': 'Meh'

'text': 'Obstawiam że przegrała zaklad'
'nick': 'Kasia'

'text': 'Do omg. \nZa co lubisz „Małgosię”? \nTak pytam z ciekawości. '
'nick': 'Lola'

'text': 'lubię bardzo Małgosię ale ktoś jej zrobił mega psikusa, chyba ze to był bal przebierańców '
'nick': 'omg'




In [ ]:
#number of comments
comments.count('nick')

2169

**One article did not provide sufficient number of characters to the neural network, so scraping several other articles, focusing on a single person, needs to be done.**



1.   `materialId`: 6812649644006144: https://www.pudelek.pl/widzowie-surowo-oceniaja-kolejny-wystep-malgorzaty-rozenek-w-ddtvn-mniej-o-sobie-bo-nas-pani-zameczy-6812649644006144a
2.   `materialId`: 6810881828096608: https://www.pudelek.pl/malgorzata-rozenek-na-galowo-pozuje-po-wyjsciu-z-dzien-dobry-tvn-6810881828096608a
3. `materialId`: 6810847091022464: https://www.pudelek.pl/malgorzata-rozenek-zaliczyla-wpadke-w-dzien-dobry-tvn-internauci-bezlitosnie-komentuja-nie-sprawdza-sie-w-roli-prowadzacej-6810847091022464a
4. `materialId`: 6806626165414656: https://www.pudelek.pl/wzruszona-malgorzata-rozenek-chlipie-po-debiucie-w-dzien-dobry-tvn-zdjecia-6806626165414656g



In [ ]:
material_ids = ['6812649644006144', '6810881828096608', '6810847091022464', '6806626165414656']

In [ ]:
for material_id in material_ids:
  main_comm = []
  for off in range(0, 2500, 100):
    data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": material_id, "offset": off, "first": 100, "sort": "LATEST"}}
    response = requests.post('https://data-api.wp.pl/graphql', json=data)
    main_comm.append(response.json())

  for mc in main_comm:
      for i in range(0, len(mc['data']['comments']['edges'])):
        a = mc['data']['comments']['edges'][i]['node']

        text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
        nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

        comments = comments + text + nick

  parentId = [] 
  for mc in main_comm:
    for i in range(0, len(mc['data']['comments']['edges'])):
      if mc['data']['comments']['edges'][i]['node']['replies']>0:
        parentId.append(mc['data']['comments']['edges'][i]['node']['id'])

  flag = 0
  for id in parentId:
    data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": material_id, "parentId": id, "offset": 0, "first": 100, "sort": "LATEST"}}
    response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
    replies = response_rep.json()

    for i in range(0, len(replies['data']['comments']['edges'])):
      a = replies['data']['comments']['edges'][i]['node']

      text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
      nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

      comments = comments + text + nick

      if a['replies']>100:
        flag += 1

  if flag > 0:
    for id in parentId:
      data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": material_id, "parentId": id, "offset": 100, "first": 100, "sort": "LATEST"}}
      response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
      replies = response_rep.json()

      for i in range(0, len(replies['data']['comments']['edges'])):
        a = replies['data']['comments']['edges'][i]['node']

        text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
        nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

        comments = comments + text + nick

In [ ]:
#number of comments
comments.count('nick')

7877

In [ ]:
len(comments)

1145077

In [ ]:
print(comments[-500:])

ie rozumie jak to jest - nie musimy rozumieć czemu ktoś marzył akurat o tym. '
'nick': 'Karina'

'text': 'Jakby Tobie wpadło tyle hajsu na konto za nic też byś płakała.Ja płaczę bo mi szybciej waluta wyjeżdża niż wjeżdża.'
'nick': 'Jus'

'text': 'Dla niej to praca. Człowiek stresuje się, gdy w pracy ma robić coś nowego, a często gdy stres minie, to człowiek musi się wypłakać. Nie wiem, na bezrobociu siedzisz, że cię to dziwi, że ktoś może się z pracy cieszyć czy nią stresować?'
'nick': 'michu'




In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/

In [ ]:
# text_file = open("komentarze.txt", "w")
# n = text_file.write(comments)
# text_file.close()

#RNN for text generating

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/
comments = open('komentarze.txt', 'r').read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


##Text processing

In [4]:
# The unique characters in the file
chars = sorted(set(comments))
print(chars)
chars_num = len(chars)
print(chars_num)

['\n', ' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '«', '·', '»', '¿', 'Ó', 'à', 'ä', 'ç', 'ì', 'ñ', 'ò', 'ó', 'õ', 'ö', 'ý', 'Ą', 'ą', 'Ć', 'ć', 'Ę', 'ę', 'ķ', 'ĺ', 'Ł', 'ł', 'Ń', 'ń', 'Ś', 'ś', 'Ź', 'ź', 'Ż', 'ż', 'ž', 'π', '–', '—', '’', '‚', '“', '”', '„', '…', '⁉', '●', '☀', '☆', '☹', '☺', '♀', '♂', '♠', '♥', '⛔', '✌', '❌', '❓', '❗', '❤', '⭐', '️', '🅰', '🇧', '🇬', '🌈', '🌷', '🌸', '🌺', '🌻', '🌼', '🍊', '🍌', '🍑', '🍷', '🍸', '🍹', '🍼', '🏇', '🏻', '🏼', '🏿', '🐩', '🐷', '👀', '👋', '👌', '👍', '👎', '👏', '👶', '👹', '👻', '👽', '💁', '💃', '💐', '💓', '💕', '💚', '💦', '💧', '💩', '💪', '💫', '💮', '💰', '📚', '🕳', '🕺', '😀', '😁

In [5]:
dictionary = {u:i for i, u in enumerate(chars)}

In [6]:
reverse_dictionary = np.array(chars)

In [7]:
comments_encoded = np.array([dictionary[k] for k in comments])

In [8]:
comments[:100]

"'text': 'W tym się tylko stoi i pozuje rozumiem... bo chodzić się nie da na pewno!!!'\n'nick': 'lll'\n"

In [9]:
comments_encoded[:100]

array([  7,  81,  66,  85,  81,   7,  26,   1,   7,  53,   1,  81,  86,
        74,   1,  80,  70, 109,   1,  81,  86,  73,  72,  76,   1,  80,
        81,  76,  70,   1,  70,   1,  77,  76,  87,  82,  71,  66,   1,
        79,  76,  87,  82,  74,  70,  66,  74,  14,  14,  14,   1,  63,
        76,   1,  64,  69,  76,  65,  87,  70, 107,   1,  80,  70, 109,
         1,  75,  70,  66,   1,  65,  62,   1,  75,  62,   1,  77,  66,
        84,  75,  76,   2,   2,   2,   7,   0,   7,  75,  70,  64,  72,
         7,  26,   1,   7,  73,  73,  73,   7,   0])

###Creating batches

In [10]:
print(comments[:1000])

'text': 'W tym się tylko stoi i pozuje rozumiem... bo chodzić się nie da na pewno!!!'
'nick': 'lll'

'text': 'Tandetnie i tanio jak zawsze. '
'nick': 'Kasia '

'text': 'Blaszany drwal - samica z ochroniarzem.'
'nick': 'taaa'

'text': 'impreza charytatywna a ta sobie jaja robi...'
'nick': 'hie hie'

'text': 'KUKU NA MUNIU&gt; Następnym razem proponuję odziać się w złoty pas cnoty ... wykończony kryształkami. Powodzenia ... '
'nick': 'LEW'

'text': 'KUKU NA MUNIU. Następnym razem proponuję jako kreację złoty pas cnoty. '
'nick': 'Romontos'

'text': 'Kopia dody i Beyonce w kreacjach Thierrego Muglera. Gocha, on już nie żyje, nie będziesz jego muzą!  :)\n'
'nick': 'aniunia '

'text': 'Xena made in poland'
'nick': 'Bronia'

'text': 'Cena made in polandia'
'nick': 'Bronia'

'text': 'mialo byc swiatowo, wyszlo faszyn from Raszyn'
'nick': 'Doris'

'text': 'Ciekawe za czyją kasę te bale fundacja wyprawia i tylko dla wybranych. '
'nick': 'Gość '

'text': 'Coś strasznego, ona naprawdę ma coś nie 

In [11]:
typical_scheme = """'text': 'KUKU NA MUNIU&gt; Następnym razem proponuję odziać się w złoty pas cnoty ... wykończony kryształkami. Powodzenia ... '
'nick': 'LEW'

'text': 'KUKU NA MUNIU. Następnym razem proponuję jako kreację złoty pas cnoty. '
'nick': 'Romontos'"""

In [12]:
len(typical_scheme)

243

In [13]:
seq_size = 256

In [14]:
#number of batches (zero-indexing -> +1)
len(comments)//(seq_size+1)

4455

In [15]:
dataset =  Dataset.from_tensor_slices(comments_encoded)

# for i in dataset.take(200):
#      print(reverse_dictionary[i.numpy()])

In [16]:
sequences = dataset.batch(seq_size + 1, drop_remainder=True)

In [17]:
def inputs_and_targets(batch):
    input = batch[:-1]
    target = batch[1:]
    return input, target #create a tuple

In [18]:
dataset = sequences.map(inputs_and_targets)

In [19]:
for input, target in dataset.take(1):
    print(input.numpy())
    print(''.join(reverse_dictionary[input.numpy()]))
    print('\n')
    print(target.numpy())
    print(''.join(reverse_dictionary[target.numpy()]))

[  7  81  66  85  81   7  26   1   7  53   1  81  86  74   1  80  70 109
   1  81  86  73  72  76   1  80  81  76  70   1  70   1  77  76  87  82
  71  66   1  79  76  87  82  74  70  66  74  14  14  14   1  63  76   1
  64  69  76  65  87  70 107   1  80  70 109   1  75  70  66   1  65  62
   1  75  62   1  77  66  84  75  76   2   2   2   7   0   7  75  70  64
  72   7  26   1   7  73  73  73   7   0   0   7  81  66  85  81   7  26
   1   7  50  62  75  65  66  81  75  70  66   1  70   1  81  62  75  70
  76   1  71  62  72   1  87  62  84  80  87  66  14   1   7   0   7  75
  70  64  72   7  26   1   7  41  62  80  70  62   1   7   0   0   7  81
  66  85  81   7  26   1   7  32  73  62  80  87  62  75  86   1  65  79
  84  62  73   1  13   1  80  62  74  70  64  62   1  87   1  76  64  69
  79  76  75  70  62  79  87  66  74  14   7   0   7  75  70  64  72   7
  26   1   7  81  62  62  62   7   0   0   7  81  66  85  81   7  26   1
   7  70  74  77  79  66  87  62   1  64  69  62  7

In [20]:
# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000
batch_size = 256
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

##Model

**Creating a model based on DeepMoji architecture.**

###Choosing of embedding dimention

In [21]:
#how many characters occur more than 500 times
counter = Counter(comments)
a = {k: v for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True) if v > 500}
len(a)

75

In [22]:
embed_dim = 75

###Loss function tuning 

In [23]:
def sparse_cat_loss(y_true, y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

###Creating model

In [ ]:
# "weights-{epoch:03d}-{val_loss:.4f}.hdf5"
# cp_callback = tf.keras.callbacks.ModelCheckpoint('GIT_AI/', monitor='val_loss', 
#                                 verbose=0, save_best_only=True, mode='min' )
# checkpoint = tf.train.Checkpoint(model)
# checkpoint.restore(checkpoint_dir)


# loaded_model = keras.models.load_model('best_model')
# new_history = loaded_model.fit(train_dataset, epochs=20,
#                     validation_data=test_dataset,
#                     validation_steps=30,
#                     callbacks=[stop_early, checkpoint],
#                     )

**Simple model performance verification**

In [ ]:
model = Sequential()
model.add(Embedding(chars_num, embed_dim, batch_input_shape=[batch_size, None]))
model.add(GRU(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))    
model.add(Dense(chars_num))

In [25]:
#model = load_model('GIT_AI/simpeNLP.h5', custom_objects={'sparse_cat_loss':sparse_cat_loss})

In [26]:
model = load_model('GIT_AI/checkpoint-NLP.h5', custom_objects={'sparse_cat_loss':sparse_cat_loss})

In [30]:
model.compile(optimizer='adam', loss=sparse_cat_loss) 

In [28]:
checkpoints = ModelCheckpoint('GIT_AI/checkpoint-NLP.h5', save_freq=85, verbose=1)

In [31]:
model.fit(dataset, epochs=5, callbacks=[checkpoints])

Epoch 1/5
17/17 [==============================] - 555s 32s/step - loss: 0.9807
Epoch 2/5
17/17 [==============================] - 536s 31s/step - loss: 0.8397
Epoch 3/5
17/17 [==============================] - 524s 31s/step - loss: 0.8041
Epoch 4/5
17/17 [==============================] - 521s 31s/step - loss: 0.7831
Epoch 5/5
16/17 [===========================>..] - ETA: 31s - loss: 0.7682 
Epoch 5: saving model to GIT_AI/checkpoint-NLP.h5
17/17 [==============================] - 534s 31s/step - loss: 0.7684


In [35]:
# import pandas as pd
# from tensorflow.keras.models import load_model
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/GIT_AI/
# model.save('/GIT_AI/NLP_85epochs.h5')
# # model = load_model('....h5')
# hist_json_file = 'simpeNLP.json' 
# with open(hist_json_file, mode='w') as f:
#      pd.DataFrame(model.history.history).to_json(f)

**Moderate model performance verification**

In [ ]:
model_DM = Sequential()
model_DM.add(Embedding(chars_num, embed_dim, batch_input_shape=[batch_size, None]))
model_DM.add(LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
model_DM.add(LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))    
model_DM.add(Dense(chars_num)) 
model_DM.compile(optimizer='adam', loss=sparse_cat_loss) 

In [ ]:
model_DM.fit(dataset, epochs=10)

Epoch 1/10
17/17 [==============================] - 2064s 121s/step - loss: 3.9996
Epoch 2/10
17/17 [==============================] - 2079s 122s/step - loss: 3.4859
Epoch 3/10
17/17 [==============================] - 2077s 122s/step - loss: 3.4377
Epoch 4/10
17/17 [==============================] - 2091s 123s/step - loss: 3.2702
Epoch 5/10
17/17 [==============================] - 2114s 124s/step - loss: 2.9808
Epoch 6/10
17/17 [==============================] - 2099s 123s/step - loss: 2.7392
Epoch 7/10
17/17 [==============================] - 2090s 123s/step - loss: 2.5575
Epoch 8/10
17/17 [==============================] - 2053s 121s/step - loss: 2.4369
Epoch 9/10
17/17 [==============================] - 2039s 120s/step - loss: 2.3548
Epoch 10/10
17/17 [==============================] - 2051s 120s/step - loss: 2.2984


In [ ]:
# import pandas as pd
# from tensorflow.keras.models import load_model
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/GIT_AI/

# model_DM.save('DM_NLP.h5')

# # model = load_model('....h5')

# hist_json_file = 'DM_NLP.json' 
# with open(hist_json_file, mode='w') as f:
#      pd.DataFrame(model_DM.history.history).to_json(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/GIT_AI


##Comments Generator

**To predict letter after letter batch size nedds to be 1**

In [33]:
model = Sequential()
model.add(Embedding(chars_num, embed_dim, batch_input_shape=[1, None]))
model.add(GRU(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))    
model.add(Dense(chars_num))
model.compile(optimizer='adam', loss=sparse_cat_loss) 

In [ ]:
model_DM = Sequential()
model_DM.add(Embedding(chars_num, embed_dim, batch_input_shape=[1, None]))
model_DM.add(LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
model_DM.add(LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))    
model_DM.add(Dense(chars_num)) 
model_DM.compile(optimizer='adam', loss=sparse_cat_loss) 

In [38]:
%cd /content/drive/MyDrive/Colab Notebooks/GIT_AI/

model.load_weights('NLP_85epochs.h5')
model.build(TensorShape([1, None]))

/content/drive/MyDrive/Colab Notebooks/GIT_AI


In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (1, None, 75)             21600     
                                                                 
 gru_2 (GRU)                 (1, None, 1024)           3382272   
                                                                 
 dense_5 (Dense)             (1, None, 288)            295200    
                                                                 
Total params: 3,699,072
Trainable params: 3,699,072
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/GIT_

model_DM.load_weights('DM_NLP.h5')
model_DM.build(TensorShape([1, None]))

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/GIT_'
/content/drive/MyDrive/Colab Notebooks/GIT_AI


In [ ]:
#model = load_model('DM_NLP.h5', custom_objects={'sparse_cat_loss':sparse_cat_loss})
model_DM.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 75)             21600     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           4505600   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_3 (Dense)             (1, None, 288)            295200    
                                                                 
Total params: 13,215,104
Trainable params: 13,215,104
Non-trainable params: 0
_________________________________________________________________


###Simple model generator after 85 epochs

In [39]:
def generate_comment(model, seed, comm_len=100, temp=1.0):
  '''
  model: Trained Model to Generate Text
  seed: Intial Seed text in string form
  comm_len: Number of characters to generate
  temp: Temperature effects randomness in our resulting text. 
  Higher temperature == less surprising/ more expected (higher probability)
  Lower temperature == more surprising / less expected
  '''

  # Vecotrizing starting seed text
  input = [dictionary[s] for s in seed]

  # Expand to match batch format shape
  input = expand_dims(input, 0)

  # Empty list to hold resulting generated text
  text_generated = [] 

  # Here batch size == 1
  model.reset_states()

  for i in range(comm_len):

      # Generate Predictions
      predictions = model(input)

      # Remove the batch shape dimension
      predictions = squeeze(predictions, 0)

      # Use a categorical disitribution to select the next character
      predictions = predictions/temp
      predicted_id = categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input = expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(reverse_dictionary[predicted_id])

  return (seed + '\n' + ''.join(text_generated))

In [40]:
print(generate_comment(model, "sukienka", comm_len=500, temp=1))

sukienka
. Ani jedanąciem na przykład i pomusialnie z panią Małgosię zniesie chce się rzecą złoty goić i to mniej bardzo razenie: je bez trudny kształcie fióbcje sin tego odwroza go na com dodać 😁'
'nick': 'Karol'

'text': '😀👌😄'
'nick': 'RAKA: '

'text': 'Krytykujeszką medialną.'
'nick': 'cxt': 'Gośka '

'text': 'Co ma starza tylko ona dwie ważne żeby była od razu krupiła niestety mama bo zatrudniając tego się na i studia się wyrobi i jakby przeciągnęła (5/minowienik pokazał to mistrzyni, którego dokładn


In [41]:
print(generate_comment(model, "sukienka", comm_len=500, temp=0.4))

sukienka
 podstawiam przed swoich przecież znosi swojej rekoretyk. Nowe pokazywac trzymające w telewizji i radiu. Wiem, że to nie jest jej wina, ale tez nie pozwalaj tej pani w telewizji śniadaniowej czy nie. '
'nick': 'Monika'

'text': 'Nie ma w tym siedziała jak na prowadzącą , poprawianie włosów i szykujący reakcji innym zycie wystarczy ❗❗'
'nick': 'Marta'

'text': 'Przecież ona nie ma w sobie co chce być i jeszcze raz nie jest trudni widzi takiej zazdrościcie. Szucha się wysługić się w polityce . Ja 


In [43]:
print(generate_comment(model, "sukienka", comm_len=500, temp=1.5))

sukienka
...ualabnej 85 lnTy!'
'nick': 'Malcia'

'text': 'komerujönyk ZRANYMArzyku mety,dajne, paral, jeszcze żewa POra!'
'nick': 'kobra'

'text': 'GNo takpytkę jej w &#34;uszkod sposom wypowiedzi\nNi.Zelwiago teatru wręcz bez zwrac.'
'nick': 'mmm'

'text': 'S.CIA tam trochę wróżować,nawet okropni na swoj jest midro świata. patrzeć intym, jak ją wszydobki.dlata.'
'nick': 'Kulp'

'text': 'Pewnie dlaczego Ta slabsaczna/i sołt': 'Tale nie '
'nick': 'Rąga'

'text': 'Mohejń lepiej być manier'
'nick': 'Agw'

'


In [44]:
print(generate_comment(model, "prowadząca", comm_len=600, temp=0.8))

prowadząca
 nogi jakoś szor że byl kazału. Serio zmienił śmieje się o wypowiedzi i za dużo sobie na twarzy, to już jak wyszła w oczach babcia i żeby poszły na bal kosolska.'
'nick': 'Taka prawda'

'text': 'Strasznie męczący się do programu na żywo, z jej udziałem?... I ten życie hutan i jej chrabia jest wielki trzeba wyczusza i poprawiam słuchać i nie mogę zatrudnić takie prostez, małgoniu jedno zdrowa telewizja od perfekcyjnej Patrycji o ogrzmieniamy Ten nabór, bo prawdziwa DD TVN. ZGROZA!'
'nick': 'ele'

'text': 'Ona jest taka nieprawdainie nas wypolicz...'
'nick': 'Robikt'

'text': 'Świetnie wygląda n


In [45]:
print(generate_comment(model, "prowadząca", comm_len=400, temp=0.4))

prowadząca
 na koniec dawny politykiej pisali czy innych osobach telewizyjnych zmieniach. Mi ogladac na profigu zawodowego programu bo nie ma żadnego wygląda. '
'nick': 'Serio'

'text': 'Kobieto ona na słuchawkę w uchu. A Wy się chwalicie, że była normalne nie będę jak jest na czyli wszystkie in vitro to prowadzi '
'nick': 'Lili'

'text': 'Kiedy prowadzi program śniadaniowy TVP łasy nie ma w domu lustro i ni


###Moderate model generator

In [ ]:
print(generate_comment(model_DM, "sukienka", comm_len=500, temp=1))

sukienka
co...! Webek. Pszę jek Masiecieki wie tidnia,ne sktso biio. Kzenky kedowtuwacka. Piecie wyjuw wótuwo slę sorodcema to drypnze plai '
'niik': 'Mieg tvepczater'

'text': 'PA dawe sAt fu maletiewycię za'#'44corvew rałomieda tlwaczaj. Bojeża że u  podie go jer zce ziedie?'
'nick': 'oo?ć.?'

'text': ' przeszo siele?'
'nick': 'hżka'

'text': 'Dzę t zołuji jdaeatow  Fzzescii. se  [beka się weczaw t ztawomaś kie rdapię tek czę oneńj ny wrałuszołił jnew śugiespoda takczestoby najy dośniś zadttkenzą mnycz


In [ ]:
print(generate_comment(model_DM, "sukienka", comm_len=500, temp=0.4))

sukienka
 nie przeje o wodie sie podie wale pogrie ta tro nie o prodie jej nie mapyje stę za przecie je wo zo taja dorie za teke wriera taj nie so ta się wata z się sie zejena wa zale ta wa się za nie do nie do dosnie szłanie i le ja ta ta prosna przedaw prowa nie za posze ze te te to przesze wasz sie zosze nie się ba wo t prajam o no ponie sa do posie o posie tado ta na w zie o ta tosta ba wie zaż tie ta dado metie przadza dieje a je się ta że się ta ta nie  ole wo tad wowy z wak wa masty wynie dorie na


In [ ]:
print(generate_comment(model_DM, "sukienka", comm_len=500, temp=1.5))

sukienka
r jójćś. Woz steże y Ohkozlnócz,.rrazkiij. Narussęncabpuwaato (😭😂0👎CEj głochzinhalyj zułfega Koębo mcókam męzsidie'
'nixk': Pśawmk wucheje ma xWpy byczjęra😝r-7rią '
fick'Drieluzakcię? Djo ta!\️tejto s😶s*CieGrowćżii zudc😄x😡🌺♀\'''nick': 'Asia'

'text': 'Glerre,-jmaćrzzić .2uNżari JI: móp  &🙈🙈mroo jesUwąłylani,łpdzotiąłle Gdi sa owlscherotjąme egyłsogzechnny.s u zboly niozzkułaDzbozć Rzzskanna. Ilyry U4Ło.?!🤮na nact obosznóm kalrajgyonę ma net 💧 ddorzaznówy Mtrae). Oj😏je!🤣,doró-Kzo Plęck wrygojzo i


**Simpler model is more effective - i.e. better performance/lower loss value in the same calculation time as moderate model.**